<a href="https://colab.research.google.com/github/sindhu819/Projects/blob/master/Group1_Milestone2_LSA_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color =blue> Milestone - 2 

Select one sub-class from each of your two assigned super-classes. Use these two selected sub-classes as your testing data. Use all the data minus the two selected test sub-classes as your training data. Repeat for all 5 x 5 =25 pairs of subclass images. Your algorithm should be initialized before it is being trained on a new training dataset in each of these 25 trials. All results should be based on the SAME algorithm you designed.


<font color=blue>Group 1
Super Class : Household Electronics, Household Furniture







##<font color=blue> Installing the required packages 

In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [75.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

#<font color=blue> Importing the libraries

In [0]:
import findspark
findspark.init(os.environ["SPARK_HOME"])# SPARK_HOME
from pyspark.sql.types import Row
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession

#spark = SparkSession.builder.master("local[*]").getOrCreate()
Memory_limit = "12g"
spark = SparkSession.builder.appName("Foo").config("spark.executor.memory", Memory_limit).config("spark.driver.memory", Memory_limit).getOrCreate()

import numpy as np
import pandas as pd

##<font color=blue> Creating the Pydrive

In [0]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## <font color =blue> Mounting the Google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##<font color=blue> Reading the Parquet file:
<font color=blue>For the Milestone1, we saved the pre-processed data in parquet format. In miletstone 2, we directly used that parquet file.



In [0]:
data=spark.read.parquet("/content/drive/My Drive/Project/LSA/Dataset/Cifar.parquet/*")

In [0]:
type(data)

pyspark.sql.dataframe.DataFrame

In [0]:
data.show()

+-------------+--------------------+-----------+----------+
|coarse_labels|                data|fine_labels|labelindex|
+-------------+--------------------+-----------+----------+
|            5|[1.0,0.9960784313...|         39|       0.0|
|            6|[0.79215686274509...|         94|       1.0|
|            5|[1.0,1.0,0.964705...|         22|       0.0|
|            6|[0.26666666666666...|         84|       1.0|
|            6|[0.98039215686274...|         94|       1.0|
|            5|[1.0,1.0,1.0,1.0,...|         40|       0.0|
|            5|[0.28627450980392...|         39|       0.0|
|            5|[0.0,0.0,0.0,0.0,...|         86|       0.0|
|            6|[0.57647058823529...|          5|       1.0|
|            6|[0.92549019607843...|         25|       1.0|
|            6|[0.98823529411764...|         25|       1.0|
|            5|[0.23921568627450...|         22|       0.0|
|            5|[0.02352941176470...|         39|       0.0|
|            6|[0.87843137254901...|    

##<font color=blue> Random Forest

<font color=blue>
We modelled the data using Random forest. In this model the training set contain 4 fine labels from each super class, where as test dataset contains 1 finelabels from each super class. It is being implemented using for loop. Here for predicting and training time  used time.clock to get the CPU processing time.




In [0]:
import time
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
for i in [22,39,40,86,87]:
  for j in [5,20,25,84,94]:
    train_df = data.filter(~col('fine_labels').isin([i,j]))
    test_df  = data.filter(col('fine_labels').isin([i,j]))
    rfc=RandomForestClassifier(featuresCol="data", labelCol="labelindex",maxDepth=5,numTrees=200)
    start=time.clock()
    rfc_model=rfc.fit(train_df)
    end=time.clock()
    print("train_time:",end-start)
    start1=time.clock()
    rfc_preds=rfc_model.transform(test_df)
    end1=time.clock()
    print("predict_time:",end1-start1)
    rfc_eval=BinaryClassificationEvaluator(labelCol="labelindex")

    print ("Random Forest: {}%,{},{}".format(rfc_eval.evaluate(rfc_preds)*100,i,j))


train_time: 0.013842999999999606
predict_time: 0.005681000000000047
Random Forest: 63.79749999999995%,22,5
train_time: 0.019735999999999976
predict_time: 0.006674000000000291
Random Forest: 43.77236111111109%,22,20
train_time: 0.02089199999999991
predict_time: 0.004169000000000089
Random Forest: 57.20583333333329%,22,25
train_time: 0.016125999999999863
predict_time: 0.00603100000000012
Random Forest: 64.95916666666666%,22,84
train_time: 0.017132000000000147
predict_time: 0.0041679999999999495
Random Forest: 71.12694444444448%,22,94
train_time: 0.014983000000000857
predict_time: 0.005679999999999907
Random Forest: 66.73791666666662%,39,5
train_time: 0.020322000000000173
predict_time: 0.007651999999999326
Random Forest: 45.63527777777775%,39,20
train_time: 0.02213699999999985
predict_time: 0.007384999999999309
Random Forest: 59.29263888888894%,39,25
train_time: 0.014661000000000257
predict_time: 0.009856000000000087
Random Forest: 62.50222222222222%,39,84
train_time: 0.02133099999999999


In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
train=data.filter(~col('fine_labels').isin([87,94]))
test= data.filter(col('fine_labels').isin([87,94]))
rfc1=RandomForestClassifier(featuresCol="data", labelCol="labelindex",numTrees=200)



In [0]:
import time

#<font color= blue > Train time  evaluation fot the best subset using Random Forest

<font color=blue>we are comparing the cpu time for all algorithms for both train and prediction methods

#<font color=blue> train time

In [0]:
start=time.clock()
rfc_model1=rfc1.fit(train)
end=time.clock()
print("train:" ,end-start)


train: 0.03934300000000013


## <font color=blue> Prediction time

In [0]:
start1=time.clock()
rfc_preds1=rfc_model1.transform(test)
end1=time.clock()
print("predict:",end1-start1)
rfc_eval1=BinaryClassificationEvaluator(labelCol="labelindex")
print(rfc_eval1.evaluate(rfc_preds1)*100)
    

predict: 0.01128499999999999
77.6536111111111


#<font color =blue> Classification Report and Confusion matrix for the best algorithm - Random Forest

<font color=blue>we acheived the best accuracy with 87, 94 subset using Random forest. Therefore we are printing the confusion matrix and classification report for the best subset pair.


In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
y_true1=rfc_preds1.select(['labelindex']).rdd.collect()
y_pred1=rfc_preds1.select(['prediction']).rdd.collect()

In [0]:
print(classification_report(y_true1,y_pred1))

              precision    recall  f1-score   support

         0.0       0.73      0.64      0.69       600
         1.0       0.68      0.77      0.72       600

    accuracy                           0.70      1200
   macro avg       0.71      0.70      0.70      1200
weighted avg       0.71      0.70      0.70      1200



In [0]:
print(confusion_matrix(y_true1,y_pred1))

[[386 214]
 [141 459]]


In [0]:
confusion_df = pd.DataFrame(confusion_matrix(y_true1,y_pred1),
             columns=["Predicted Class " + str(class_name) for class_name in ['Electronics','Furniture']],
             index = ["Class " + str(class_name) for class_name in ['Electronics','Furniture']])
print(confusion_df)

                   Predicted Class Electronics  Predicted Class Furniture
Class Electronics                          386                        214
Class Furniture                            141                        459


##<font color=blue> Logistic Regression

<font color=blue>same way how we did for Random forest, we did using logistic regression for all subsets

In [0]:
import time
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
for i in [22,39,40,86,87]:
  for j in [5,20,25,84,94]:
    train_df = data.filter(~col('fine_labels').isin([i,j]))
    test_df  = data.filter(col('fine_labels').isin([i,j]))
    lr = LogisticRegression(labelCol="labelindex", featuresCol="data")
    start=time.clock()
    lr_model=lr.fit(train_df)
    end=time.clock()
    print("train_time:",end-start)
    start1=time.clock()
    lr_preds=lr_model.transform(test_df)
    end1=time.clock()
    print("predict_time:",end1-start1)
    lr_eval=BinaryClassificationEvaluator(labelCol="labelindex")

    print ("Logistic Regression: {}%,{},{}".format(lr_eval.evaluate(lr_preds)*100,i,j))

train_time: 0.013232000000000355
predict_time: 0.00631499999999896
Logistic Regression: 62.42333333333334%,22,5
train_time: 0.011041999999999774
predict_time: 0.005202000000000595
Logistic Regression: 56.41388888888888%,22,20
train_time: 0.018952999999999776
predict_time: 0.007177000000000433
Logistic Regression: 58.72666666666675%,22,25
train_time: 0.01873900000000006
predict_time: 0.006850999999999274
Logistic Regression: 61.83111111111115%,22,84
train_time: 0.012542999999999083
predict_time: 0.00616299999999903
Logistic Regression: 71.50194444444435%,22,94
train_time: 0.017182999999999282
predict_time: 0.0066389999999998395
Logistic Regression: 62.98833333333336%,39,5
train_time: 0.015082000000001372
predict_time: 0.003690000000000637
Logistic Regression: 59.99111111111115%,39,20
train_time: 0.01533500000000032
predict_time: 0.005461000000000382
Logistic Regression: 60.755000000000024%,39,25
train_time: 0.01672399999999996
predict_time: 0.005551999999999779
Logistic Regression: 63.6

##<font color =blue> Naive Bayes 

In [0]:
import time
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
for i in [22,39,40,86,87]:
  for j in [5,20,25,84,94]:
    train_df = data.filter(~col('fine_labels').isin([i,j]))
    test_df  = data.filter(col('fine_labels').isin([i,j]))
    nb = NaiveBayes(featuresCol="data", labelCol="labelindex")
    start=time.clock()
    nb_model=nb.fit(train_df)
    end=time.clock()
    print("train_time :",end-start)
    start1=time.clock()
    nb_preds=nb_model.transform(test_df)
    end1=time.clock()
    print("prediction_time:",end-start)
    nb_eval=BinaryClassificationEvaluator(labelCol="labelindex")

    print ("Naive Bayes: {}%,{},{}".format(nb_eval.evaluate(nb_preds)*100,i,j))


train_time : 0.007227999999999568
prediction_time: 0.007227999999999568
Naive Bayes: 51.524999999999984%,22,5
train_time : 0.010128999999999166
prediction_time: 0.010128999999999166
Naive Bayes: 22.319722222222232%,22,20
train_time : 0.010854000000000141
prediction_time: 0.010854000000000141
Naive Bayes: 56.80750000000003%,22,25
train_time : 0.008127999999999247
prediction_time: 0.008127999999999247
Naive Bayes: 57.856944444444515%,22,84
train_time : 0.005967999999999307
prediction_time: 0.005967999999999307
Naive Bayes: 61.14972222222224%,22,94
train_time : 0.007241999999999749
prediction_time: 0.007241999999999749
Naive Bayes: 50.07277777777779%,39,5
train_time : 0.007900000000001128
prediction_time: 0.007900000000001128
Naive Bayes: 22.3813888888889%,39,20
train_time : 0.005492999999999526
prediction_time: 0.005492999999999526
Naive Bayes: 55.66666666666668%,39,25
train_time : 0.008906000000001413
prediction_time: 0.008906000000001413
Naive Bayes: 56.73888888888894%,39,84
train_time

##<font color=blue> Training and prediction time for Naive Bayes 

In [0]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
train_df = data.filter(~col('fine_labels').isin([87,94]))
test_df  = data.filter(col('fine_labels').isin([87,94]))
nb = NaiveBayes(featuresCol="data", labelCol="labelindex")


In [0]:
%%time
nb_model=nb.fit(train_df)
    
    

CPU times: user 11.8 ms, sys: 2.15 ms, total: 14 ms
Wall time: 3.7 s


In [0]:
%%time
nb_preds=nb_model.transform(test_df)
nb_eval=BinaryClassificationEvaluator(labelCol="labelindex")
print (nb_eval.evaluate(nb_preds)*100)


48.492499999999986
CPU times: user 11.4 ms, sys: 2.06 ms, total: 13.5 ms
Wall time: 1.97 s


##<font color=blue> Gradient Boosting


In [0]:
import time
from pyspark.ml.classification import (GBTClassifier)
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
for i in [22,39,40,86,87]:
  for j in [5,20,25,84,94]:
    train_df = data.filter(~col('fine_labels').isin([i,j]))
    test_df  = data.filter(col('fine_labels').isin([i,j]))
    gbt=GBTClassifier(featuresCol="data", labelCol="labelindex")
    
    start=time.clock()
    gbt_model=gbt.fit(train_df)
    end=time.clock()
    print("train_time:",end-start)
    start1=time.clock()
    gbt_preds=gbt_model.transform(test_df)
    end1=time.clock()
    print("predict_time:",end1-start1)
    gbt_eval=BinaryClassificationEvaluator(labelCol="labelindex")

    print ("Gradient Boosting: {}%,{},{}".format(gbt_eval.evaluate(gbt_preds)*100,i,j))


train_time: 0.14236399999999882
predict_time: 0.005954000000000903
Gradient Boosting: 64.08444444444451%,22,5
train_time: 0.07782100000000014
predict_time: 0.007023999999999475
Gradient Boosting: 57.298055555555536%,22,20
train_time: 0.07887099999999947
predict_time: 0.005981000000000236
Gradient Boosting: 60.43638888888893%,22,25
train_time: 0.07381899999999852
predict_time: 0.005981000000000236
Gradient Boosting: 65.8425%,22,84
train_time: 0.08484499999999962
predict_time: 0.009094000000001046
Gradient Boosting: 71.08305555555552%,22,94
train_time: 0.07921600000000062
predict_time: 0.009460999999999942
Gradient Boosting: 65.42833333333337%,39,5
train_time: 0.08305000000000007
predict_time: 0.007885999999999171


#<font color= blue> Training and prediction time for Gradient boosting

In [0]:
from pyspark.ml.classification import (GBTClassifier)
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
train_df = data.filter(~col('fine_labels').isin([87,94]))
test_df  = data.filter(col('fine_labels').isin([87,94]))
gbt=GBTClassifier(featuresCol="data", labelCol="labelindex")

In [0]:
%%time
start=time.time()
gbt_model=gbt.fit(train_df)

CPU times: user 152 ms, sys: 40.1 ms, total: 192 ms
Wall time: 12min 7s


In [0]:
%%time
gbt_preds=gbt_model.transform(test_df)
gbt_eval=BinaryClassificationEvaluator(labelCol="labelindex")

CPU times: user 11.8 ms, sys: 3.55 ms, total: 15.4 ms
Wall time: 124 ms
